In [2]:
!git clone https://github.com/cd-public/Isadora.git

Cloning into 'Isadora'...
remote: Enumerating objects: 5272, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 5272 (delta 5), reused 9 (delta 5), pack-reused 5260 (from 1)
Receiving objects: 100% (5272/5272), 106.03 MiB | 19.90 MiB/s, done.
Resolving deltas: 100% (4388/4388), done.


In [3]:
ls Isadora/model/single/vcds

ACLK.vcd                       M_AXI_RUSER_wire.vcd
ARESETN.vcd                    M_AXI_RVALID.vcd
AR_ADDR_VALID.vcd              M_AXI_RVALID_wire.vcd
AR_ADDR_VALID_FLAG.vcd         M_AXI_WDATA.vcd
AR_CH_DIS.vcd                  M_AXI_WDATA_wire.vcd
AR_CH_EN.vcd                   M_AXI_WLAST.vcd
AR_EN_RST.vcd                  M_AXI_WLAST_wire.vcd
AR_HIGH_ADDR.vcd               M_AXI_WREADY.vcd
AR_ILLEGAL_REQ.vcd             M_AXI_WREADY_wire.vcd
AR_ILL_TRANS_FIL_PTR.vcd       M_AXI_WSTRB.vcd
AR_ILL_TRANS_SRV_PTR.vcd       M_AXI_WSTRB_wire.vcd
AR_STATE.vcd                   M_AXI_WUSER.vcd
AW_ADDR_VALID.vcd              M_AXI_WUSER_wire.vcd
AW_ADDR_VALID_FLAG.vcd         M_AXI_WVALID.vcd
AW_CH_DIS.vcd                  M_AXI_WVALID_wire.vcd
AW_CH_EN.vcd                   R_STATE.vcd
AW_EN_RST.vcd                  S_AXI_CTRL_ARADDR.vcd
AW_HIGH_ADDR.vcd               S_AXI_CTRL_ARPROT.vcd
AW_ILLEGAL_REQ.vcd             S_AXI_CTRL_ARREADY.vcd
AW_ILL_DATA_TRANS_SRV_PTR.vcd  S_AXI_CTRL_ARVA

In [4]:
# adapted from https://github.com/vcd2df/py
# use a generator over a split (still lazy) instead of file ptr

# on my device, /home/user/.local/bin/spark-submit script.py

import pandas as pd
import pickle
from pathlib import Path

def get_vars(lines):
    line = next(lines)
    vars = {}
    while "$enddefinitions" not in line:
        if "var" in line:
            parts = line.split()
            if parts[4] not in vars.values():
                vars[parts[3]] = parts[4]
        line = next(lines)
    return vars

def str2df(vcd_str):
    lines = (line for line in vcd_str.splitlines())
    vars = get_vars(lines)
    names = vars.copy()
    vars = {var: -1 for var in vars.keys()}
    df = {}
    while "$dumpvars" not in next(lines):
        pass
    time = "#0"
    for line in lines:
        if line.startswith("#"):
            df[time] = pd.Series(vars.values())
            time = line.strip()
        else:
            if " " in line:
                val, var = line[1:].strip().split()
            else:
                val, var = line[0], line[1:].strip()
            if var in vars:
                vars[var] = int(val, 2) if val.isdigit() else -1
    df = pd.DataFrame(df, dtype=int)
    df.index = names.values()
    return df

def iflow_times(local):
    local = local[local.index.str.contains("shadow")]
    local = local[local.any(axis=1)]
    local = local.idxmax(axis=1)
    local = local[local != "#0"]
    local = local.apply(lambda s: int(s[1:]))
    return local

def iflow_times2(local):
    local = local[local.index.str.contains("shadow")]
    local = local[local.any(axis=1)]
    local = local.eq(local.max(axis=1), axis=0)
    local = local.apply(lambda row: row[row].index.tolist(), axis=1)
    local = local.apply(lambda s : int(s[0][1:]) if isinstance(s, list) else int(s[1:]))
    return local

def iflow_times3(local):
    max_cols = local.idxmax(axis=1)
    corrupted_rows = max_cols[max_cols == "#0"]
    if corrupted_rows.empty:
        return pd.Series(dtype=int)
    #'#0' to 0
    result = corrupted_rows.apply(lambda s: int(s[1:]))
    return result

def process_vcd_folder(folder_path):
    folder = Path(folder_path)
    result = []
    for file in folder.glob("*.vcd"):
        vcd_text = file.read_text()
        src_name = file.stem.replace("shadow_", "")
        df = str2df(vcd_text)
        flow = iflow_times2(df)
        if not flow.empty:
            result.append({src_name: flow})
    return result

results = process_vcd_folder( "Isadora/model/single/vcds" )

# Save to pickle file
output_path = "iflow_times2_single.pkl"
with open(output_path, "wb") as f:
    pickle.dump(results, f)

print(f"Results saved to {output_path}")

Results saved to iflow_times2_single.pkl
